![logo](https://docs.conan.io/en/latest/_static/conan_logo.png)

# Welcome to conan

Conan is a portable package manager, intended for C and C++ developers, but it is able to manage builds from source, dependencies, and precompiled binaries for any language.

For more information, check [conan.io](conan.io).

Install
=======

Conan can be installed in many Operating Systems. It has been extensively used and tested in Windows, Linux (different distros), OSX, and is
also actively used in FreeBSD and Solaris SunOS. There are also several additional operating systems on which it has been reported to work.

There are three ways to install conan:

1. The preferred and **strongly recommended way to install Conan** is from PyPI, the Python Package Index, using the ``pip`` command.
2. There are other available installers for different systems, which might come with a bundled python interpreter, so that you don't have to
   install python first. Please note that some of **these installers might have some limitations**, specially those created with pyinstaller
   (such as Windows exe & Linux deb).
3. Running conan from sources.

Install with pip (recommended)
------------------------------

To install Conan using **pip**, you need a python 2.7 or 3.X distribution installed in your machine. Modern python distros come 
with pip pre-installed. However, if necessary you can install pip by following the instructions in `pip docs`.

Install conan:

In [ ]:
!pip install conan

Initial configuration
---------------------

Let's check if conan is correctly installed. In your console, run the following:

In [ ]:
!conan

Getting started
===============

Let's start with an example using one of the most popular C++ libraries: [POCO](https://pocoproject.org/). For convenience purposes we'll use CMake. Keep in mind that
Conan **works with any build system** and does not depend on CMake.


A Timer using POCO libraries
----------------------------

If your code is in a GitHub repository you can simply clone the project, instead of creating this folder, using the following command:

In [ ]:
!git clone https://github.com/memsharded/example-poco-timer.git

Next, let's check the following source files inside this folder:

In [ ]:
!pygmentize -g example-poco-timer/timer.cpp

Now let's read a conanfile.txt inside this folder with the following content:

In [ ]:
!pygmentize -g example-poco-timer/conanfile.txt

In this example we will use CMake to build the project, which is why the ``cmake`` generator is specified. This generator will create a
*conanbuildinfo.cmake* file that defines CMake variables as include paths and library names, that can be used in our build.

If you are not a CMake user, change the ``[generators]`` section of your *conanfile.txt* to ``gcc`` or a more generic one ``txt`` to handle requirements with any build system.

Just include the generated file and use these variables inside our *CMakeLists.txt*:

In [ ]:
!pygmentize -g example-poco-timer/CMakeLists.txt

Installing dependencies
-----------------------

Then create a build folder, for temporary build files, and install the requirements (pointing to the parent directory, as it is where the *conanfile.txt* is):

In [ ]:
!conan install example-poco-timer

This `conan install` command will download the binary package required for your configuration (detected the first time  that you ran the command), **together with other (transitively required by Poco) libraries, like OpenSSL and Zlib**.

It will also create the *conanbuildinfo.cmake* file in the current directory, in which you can see the cmake defined variables, and a *conaninfo.txt* where information about settings, requirements and options is saved.

It is very important to understand the installation process. When `conan install` command is issued, it will use some settings, specified on the command line or taken from the defaults in ``<userhome>/.conan/profiles/default`` file.

![install_flow](https://docs.conan.io/en/latest/_images/install_flow.png)

For a command like `conan install . -s os="Linux" -s compiler="gcc"`, the steps are:

- Check if the package recipe (for ``Poco/1.8.0.1@pocoproject/stable`` package) exists in the local cache. If we are just starting, the
  cache will be empty.
- Look for the package recipe in the defined remotes. Conan comes with `conan-center`_ Bintray remote by default (you can change that).
- If the recipe exists, Conan client will fetch and store it in your local cache.
- With the package recipe and the input settings (Linux, gcc), Conan client will check in the local cache if the corresponding binary is
  there, if we are installing for the first time, it won't.
- Conan client will search for the corresponding binary package in the remote, if it exists, it will be fetched.
- Conan client will then finish generating the requested files specified in ``generators``.

If the binary package necessary for some given settings doesn't exist, Conan client will throw an error. It is possible to try to build the
binary package from sources with the ``--build=missing`` command line argument to install. A detailed description of how a binary package is
built from sources will be given in a later section.

Building the timer example
--------------------------

Now you are ready to build and run your project:

In [ ]:
!cmake example-poco-timer
!cmake --build .

In [ ]:
!bin/timer

Inspecting dependencies
-----------------------

The retrieved packages are installed to your local user cache (typically ``.conan/data``), and can be reused from there in other projects.
This allows to clean your current project and keep working even without network connection. Search packages in the local cache using:

In [ ]:
!conan search

Inspect binary package details (for different installed binaries for a given package recipe) using:

In [ ]:
!conan search Poco/1.8.0.1@pocoproject/stable

There is also the option to generate a table for all binaries from a given recipe with the ``--table`` option, even in remotes:

In [ ]:
!conan search Poco/1.8.0.1@pocoproject/stable --table=file.html -r=conan-center
from IPython.display import HTML
HTML(filename="./file.html")

Check the reference for more information on how to search in remotes, how to remove or clean packages from the local cache, and how to
define custom cache directory per user or per project.

Inspect your current project's dependencies with the ``info`` command, pointing it to the folder where the *conanfile.txt* is:

In [ ]:
!conan info example-poco-timer

Generate a graph of your dependencies in dot or html formats:

In [ ]:
!conan info example-poco-timer --graph=info.html
from IPython.display import HTML
HTML(filename="./info.html")

Searching packages
------------------
The packages that have been used are installed from the remote repository that is configured by default in the conan client, which is called "conan-center" and is in Bintray. You can search for existing packages there with:

In [ ]:
!conan search "Poco*" -r=conan-center

There are other community repositories that can be configured and used.


Building with other configurations
----------------------------------

In this example we have built our project using the default configuration detected by conan, this configuration is known as the `default profile`.

The first time you run the command that requires a profile, such as `conan install`, your settings are detected (compiler, architecture...) automatically and stored as default in a profile. You can change your those settings by editing ``~/.conan/profiles/default`` or create new profiles with the desired configuration.

For example, if we have a profile with a gcc configutarion for 32 bits in a profile called *gcc_x86*, we could issue the ``install``.

However, the user can always override the default profile settings in ``install`` command with the ``-s`` parameter. As an exercise, try building your timer project with a different configuration. For example, you could try building the 32 bits version:

In [ ]:
!conan install example-poco-timer -s arch=x86

This will install a different package, using the ``-s arch=x86`` setting, instead of the default used previously, that in most cases will be
``x86_64``.

To use the 32 bits binaries you will also have to change your project build:

- In Windows, change the invocation accordingly to ``Visual Studio 14``.
- In Linux, you have to add the ``-m32`` flag to your ``CMakeLists.txt`` with ``SET(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS}-m32")``, and the same to ``CMAKE_C_FLAGS, CMAKE_SHARED_LINK_FLAGS and CMAKE_EXE_LINKER_FLAGS``. This can also be done more easily, automatically with Conan, as we'll see later.
- In Mac, you need to add the definition ``-DCMAKE_OSX_ARCHITECTURES=i386``.